In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
%cd gdrive/MyDrive/Anime_Recommender/Greta_CE

/content/gdrive/MyDrive/Anime_Recommender/Greta_CE


In [ ]:
!pip install sentence_transformers

In [71]:
from sentence_transformers import SentenceTransformer, SentencesDataset, util
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
import cross_encoder_train 
import bi_encoder_train 
import recommender
import os
import csv
import pickle
import time
import sys
import pandas as pd
import torch
from torch.utils.data import DataLoader
import math
import gzip
from zipfile import ZipFile
import random
from sentence_transformers import SentencesDataset, losses
from sentence_transformers.evaluation import BinaryClassificationEvaluator
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [76]:
#Movie lens data - TODO: Get code to work without truncation. Truncation set to 1000 for now
f1='ml-20m//movies.csv'
f2='ml-20m//ratings.csv'
f3='ml-20m//genome-scores.csv'
f4='ml-20m//genome-tags.csv'
f5='ml-20m//tags.csv'
data=recommender.movie_lens_data(f1,f2,f3,f4,f5, truncate=True) 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/content/gdrive/MyDrive/Anime_Recommender/Greta_CE/recommender.py:43: FutureWarning: The default value of regex will change from True to False in a future 

In [77]:
data.bert_tokenize()

In [78]:
user_keywords = "murder bloody"


recommendations = data.get_recommendations_by_keyword_bert_tfidf(user_keywords, num_recommendations=5, bert_weight=.5, tfidf_weight=.5)
print(recommendations)

['Dead Man Walking (1995)', 'Cry, the Beloved Country (1995)', 'Postman, The (Postino, Il) (1994)', 'Misérables, Les (1995)', 'In the Bleak Midwinter (1995)']


In [79]:
# Configure and train Cross-Encoder using Quora Sentence Pairs dataset. Use the filtered 500 synopses as inout to CrossEncoder. A CrossEncoder gets both inputs (input request, synopsis) and outputs similarity score
cross_encoder_model = cross_encoder_train.CE_Model()
cross_encoder_model.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nreimers/MiniLM-L6-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Configure BiEncoder (SentenceTransformer) that produces embeddings for input request and uses cosine similarity to filter top 500 similar synposes.
bi_encoder_model = bi_encoder_train.BE_Model()
num_candidates = 500


In [53]:
# Dataset we use for BE training. Old - not in use, but current BE trained on this data (with labesl from CE)
# dataset_path= "./data/movies_large.csv"

#Data used for recommender (smaller dataset)
dataset_path = "./data/movies_kaggle_merged.csv"

In [55]:
#Check if dataframe exists
df_cache_path = "df_small.pkl"
if os.path.exists(df_cache_path):
    print("Load pre-computed df from disc")
    df = pd.read_pickle("./df_small.pkl")  
else:
    df = pd.read_csv(dataset_path)
    df.dropna(subset=['overview'], inplace=True)
    df.drop_duplicates(subset='overview', keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)

    print("Store df file on disc")
    df.to_pickle("./df_small.pkl")  

Load pre-computed df from disc


In [56]:
#Prepare CE output to train Bi-Encoder Model
train_num_samples = 10000 #Can adjust this number
qid = [] #query id
train_sentence_pairs = [] 
df_train = df[:train_num_samples]
# print(df_train)
for i in range(train_num_samples):
    rand_num = random.randint(0,train_num_samples-1)
    sentence_1=df_train.iloc[rand_num, 4]
    rand_num = random.randint(0,train_num_samples-1)
    sentence_2=df_train.iloc[rand_num, 4]
    train_sentence_pairs.append([sentence_1, sentence_2])

train_ce_scores = cross_encoder_model.model.predict(train_sentence_pairs)
binary_train_ce_scores=[1 if score>=.2 else 0 for score in train_ce_scores] #Can tune score threshold




In [57]:
#Train Bi-Encoder using Cross-Encoder output and Quora Sentence Pairs dataset
bi_encoder_model.train(binary_train_ce_scores, train_sentence_pairs)

In [59]:
# File for caching  embeddings
embedding_cache_path = 'movies-summary-embeddings-size-small.pkl'
embedding_cache_path

'movies-summary-embeddings-size-small.pkl'

In [61]:
#Check if embedding cache path exists

if os.path.exists(embedding_cache_path):
    with open(embedding_cache_path, "rb") as fIn:
        cached_data = pickle.load(fIn)
        corpus_sentences = cached_data['corpus_sentences']
        corpus_embeddings = cached_data['corpus_embeddings']
else:
    corpus_sentences = df['overview'].tolist()
    corpus_embeddings = bi_encoder_model.model.encode(corpus_sentences, show_progress_bar=True, convert_to_tensor=True)   
    with open(embedding_cache_path, "wb") as fOut:
        pickle.dump({'corpus_sentences': corpus_sentences, 'corpus_embeddings': corpus_embeddings}, fOut)


    

In [58]:
print(len(df))

14104


In [62]:
print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

Corpus loaded with 14104 sentences / embeddings


In [84]:
while True:
    inp_request = input("Please enter a request: ")
    print(f"Input request: {inp_request}")

    #TODO - Add Rob's code here. Not working with current dataset
    
    #First, filter 500 candidates using bi-encoder
    start_time = time.time()
    question_embedding = bi_encoder_model.model.encode(inp_request, convert_to_tensor=True)
    
    #Get top 5 results from bi-encoder
    encoder_res = util.semantic_search(question_embedding, corpus_embeddings, top_k=num_candidates)[0]

    print(f"Cosine-Similarity search took {(time.time()-start_time):.3f} seconds")
    print("Top 5 hits with cosine-similarity:")
    for res in encoder_res[0:5]:
        print(f"\t{(res['score']):.3f}\t{df['title'].iloc[res['corpus_id']]}")

    #Use filtered results form bi-encoder as input to cross-encoder
    start_time = time.time()
    sentence_pairs = [[inp_request, corpus_sentences[res['corpus_id']]] for res in encoder_res] #pair all summaries with input request
    ce_scores = cross_encoder_model.model.predict(sentence_pairs)


    for i in range(len(encoder_res)):
        encoder_res[i]['cross-encoder_score'] = ce_scores[i]

    #Sort CrossEncoder output results by scores
    encoder_res = sorted(encoder_res, key=lambda x: x['cross-encoder_score'], reverse=True)
    print(f"\nRe-ranking with Cross-Encoder took {(time.time()-start_time):.3f} seconds")
    print("Top 5 hits with CrossEncoder:")
    for res in encoder_res[0:5]:
        print(f"\t{(res['cross-encoder_score']):.3f}\t{df['title'].iloc[res['corpus_id']]}")
    print("\n\n========\n")

Please enter a request: Alice falls down the hole
Input request: Alice falls down the hole
Cosine-Similarity search took 0.014 seconds
Top 5 hits with cosine-similarity:
	0.527	Alice Doesn't Live Here Anymore
	0.506	Resident Evil: Afterlife
	0.496	Paris-Manhattan
	0.479	Resident Evil: Apocalypse
	0.476	Alice

Re-ranking with Cross-Encoder took 0.265 seconds
Top 5 hits with CrossEncoder:
	0.016	Baby Take a Bow
	0.008	Pocketful of Miracles
	0.006	Beloved
	0.006	Snow White
	0.005	Out Cold



Please enter a request: Alice falls down the rabbit hole
Input request: Alice falls down the rabbit hole
Cosine-Similarity search took 0.013 seconds
Top 5 hits with cosine-similarity:
	0.621	Alice in Wonderland
	0.578	Alice in Wonderland
	0.520	Alice in Murderland
	0.511	Dreamchild
	0.510	Paris-Manhattan

Re-ranking with Cross-Encoder took 0.280 seconds
Top 5 hits with CrossEncoder:
	0.005	Alice in Wonderland
	0.005	Beloved
	0.005	Pooh's Grand Adventure: The Search for Christopher Robin
	0.004	The Sev

KeyboardInterrupt: ignored